In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import scipy.fftpack as fft


In [4]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 6, 7],
    [4, 5, 6, 7, 8]])


In [18]:
def convolveDesacopladaDot(img, kernelx, kernely):

    kernelx = np.flip(kernelx)
    kernely = np.flip(kernely)

    nrows, ncols = img.shape
    output = np.zeros((nrows, ncols))

    for i in range(nrows):
      output[i, :] = convolveDot(img[i, :], kernelx)
    # print(output)
    output2 = np.zeros((nrows, ncols))

    for j in range(ncols):
      output2[:, j] = convolveDot(output[:, j], kernely)
    # print(output2)
    return output2


def convolveDot(signal, kernel):
    ker_size = kernel.size
    sig_size = signal.size
    # kernel = kernel[::-1]
    pad_size = (ker_size - 1) // 2
    padded_signal = np.pad(signal, pad_size, mode='wrap')
    output = np.zeros(sig_size)
    output = [np.dot(padded_signal[k:k+ker_size], kernel)
              for k in range(sig_size)]
    return output


def completeConvolution(img, kernel):
    kernel = np.flip(kernel)
    nrows, ncols = img.shape
    krows, kcols = kernel.shape
    pad_size = (krows - 1) // 2
    padded_img = np.pad(img, pad_size, mode='wrap')
    output = np.zeros((nrows, ncols))
    for i in range(nrows):
        for j in range(ncols):
            output[i, j] = np.sum(
                padded_img[i:i+krows, j:j+kcols] * kernel)
    return output


def SuavizadoIntegral(imgIntegral, boxSize):
    N = 2*boxSize+1
    nrows, ncols = imgIntegral.shape
    pad = (N - 1) // 2
    integralSuavizada = np.zeros((nrows, ncols))

    for r in range(nrows):
        for c in range(ncols):
            r0 = r - pad
            c0 = c - pad
            r1 = r + pad
            c1 = c + pad

            if r1 >= nrows:
                r1 = nrows - 1
            if c1 >= ncols:
                c1 = ncols - 1

            s0 = imgIntegral[r1, c1]
            s1 = 0
            s2 = 0
            s3 = 0

            if r0 > 0:
                s1 = imgIntegral[r0-1, c1]
            if c0 > 0:
                s2 = imgIntegral[r1, c0-1]
            if r0 > 0 and c0 > 0:
                s3 = imgIntegral[r0-1, c0-1]
            integralSuavizada[r, c] = s0 - s1 - s2 + s3
    return integralSuavizada/(N*N)


def integralImg(img):
    nr = img.shape[0]
    nc = img.shape[1]
    intImg = np.zeros((nr, nc))
    intImg = np.cumsum(np.cumsum(img, axis=0), axis=1)
    return intImg



In [6]:
np.pad(b, 2, mode='wrap')

array([[6, 7, 3, 4, 5, 6, 7, 3, 4],
       [7, 8, 4, 5, 6, 7, 8, 4, 5],
       [4, 5, 1, 2, 3, 4, 5, 1, 2],
       [5, 6, 2, 3, 4, 5, 6, 2, 3],
       [6, 7, 3, 4, 5, 6, 7, 3, 4],
       [7, 8, 4, 5, 6, 7, 8, 4, 5],
       [4, 5, 1, 2, 3, 4, 5, 1, 2],
       [5, 6, 2, 3, 4, 5, 6, 2, 3]])

In [7]:
convolveDesacopladaDot(b, a, a)

array([[1140., 1065.,  915., 1065., 1140.],
       [1125., 1050.,  900., 1050., 1125.],
       [ 990.,  915.,  765.,  915.,  990.],
       [1095., 1020.,  870., 1020., 1095.]])

In [8]:
x = np.array([[7, 8, 1, 4, 2],
              [10, 11, 5, 3, 4],
              [1, 20, 6, 9, 12],
              [13, 1, 15, 14, 0]])
h = np.array(
    [[1, 4, 6, 4, 1],
     [2, 8, 12, 8, 2],
     [1, 4, 6, 4, 1],
     ])
h1 = np.array(
    [1, 4, 6, 4, 1])
h2 = np.array([1,2,1])

In [9]:
from scipy.signal import convolve2d

scipi = convolve2d(x, h, mode='same', boundary='wrap')
print(scipi)


[[413 437 389 335 346]
 [492 521 428 348 387]
 [537 603 593 527 492]
 [458 519 554 514 451]]


In [10]:
convolveDesacopladaDot(x, h1, h2)

array([[413., 437., 389., 335., 346.],
       [492., 521., 428., 348., 387.],
       [537., 603., 593., 527., 492.],
       [458., 519., 554., 514., 451.]])

In [12]:
h = np.array([
    [1, 2, 1],
    [1, 2, 1],[1, 2, 1]
])
completeConvolution(x, h)

array([[ 86.,  91.,  83.,  69.,  63.],
       [ 93., 108.,  79.,  62.,  70.],
       [ 96., 114., 110.,  94.,  82.],
       [ 85., 101., 100.,  90.,  76.]])

In [25]:
x = np.array([[1, 2, 3, 4, 5],
              [6, 7, 8, 9, 10],
              [11, 12, 13, 14, 15],
              [16, 17, 18, 19, 20],
              [21, 22, 23, 24, 25]])
h = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
])
convolve2d(x, h, mode='same', boundary='wrap')


array([[ 99,  93, 102, 111, 105],
       [ 69,  63,  72,  81,  75],
       [114, 108, 117, 126, 120],
       [159, 153, 162, 171, 165],
       [129, 123, 132, 141, 135]])

In [14]:
# remove the for loops of the following function






In [17]:
x = np.array([[1, 2, 3, 4, 5],
              [6, 7, 8, 9, 10],
              [11, 12, 13, 14, 15],
              [16, 17, 18, 19, 20],
              [21, 22, 23, 24, 25]])

integralImg(x)

array([[  1,   3,   6,  10,  15],
       [  7,  16,  27,  40,  55],
       [ 18,  39,  63,  90, 120],
       [ 34,  72, 114, 160, 210],
       [ 55, 115, 180, 250, 325]])

In [26]:
y = np.ones((3, 3))

x = np.array([[1, 2, 3, 4, 5],
              [6, 7, 8, 9, 10],
              [11, 12, 13, 14, 15],
              [16, 17, 18, 19, 20],
              [21, 22, 23, 24, 25]])

aux = np.zeros((5, 5))
aux[0, 0] = 1

y = completeConvolution(aux, y)

YFFT = fft.fft2(y)
XFFT = fft.fft2(x)

IFFT = fft.ifft2(YFFT*XFFT)

print(IFFT.real)


[[ 99.  93. 102. 111. 105.]
 [ 69.  63.  72.  81.  75.]
 [114. 108. 117. 126. 120.]
 [159. 153. 162. 171. 165.]
 [129. 123. 132. 141. 135.]]
